In [18]:
from pre_process import PreProcess
from word_to_vector import WordToVector
from one_hot_encoding import OneHotEncoder
import numpy as np
from training import DnnTraining
from inference import DnnInference
from keras.models import load_model

In [20]:
word_arr_one, label_arr_one = PreProcess.getTrainingTuple(dataFile='../../data/en-train.conll', onlyBioTagging=True)
word_arr_two, label_arr_two = PreProcess.getTrainingTuple(dataFile='../../data/en-dev.conll', onlyBioTagging=True)

word_arr = word_arr_one + word_arr_two
label_arr = label_arr_one + label_arr_two

n = len(word_arr)
train_split = int(0.7 * n)
val_split = int(0.2 * n)
train_word_arr = word_arr[:train_split]
val_word_arr = word_arr[train_split:train_split + val_split]
test_word_arr = word_arr[train_split + val_split:]
print(len(train_word_arr), len(val_word_arr), len(test_word_arr))

m = len(label_arr)
train_label_split = int(0.7 * m)
val_label_split = int(0.2 * m)
train_label_arr = label_arr[:train_label_split]
val_label_arr = label_arr[train_label_split:train_label_split + val_label_split]
test_label_arr = label_arr[train_label_split + val_label_split:]
print(len(train_label_arr), len(val_label_arr), len(test_label_arr))


186433 53266 26635
186433 53266 26635


In [ ]:
def pipelineTest(train_word_arr, train_label_arr, val_word_arr, val_label_arr):
    # Get wordToVector from [wordArr] and oneHotEncoding from [labelArr]
    wordToVecArr = WordToVector.getPretrainedWordToVecList(train_word_arr)
    oneHotEncodingArr = OneHotEncoder.getOneHotEncodingOfOutput(train_label_arr)
    # Convert python array to num py array
    np_wordToVecArr = np.array(wordToVecArr)
    np_oneHotEncodingArr = np.array(oneHotEncodingArr)

    # Get wordToVector from [wordArr] and oneHotEncoding from [labelArr]
    val_wordToVecArr = WordToVector.getPretrainedWordToVecList(val_word_arr)
    val_oneHotEncodingArr = OneHotEncoder.getOneHotEncodingOfOutput(val_label_arr)
    # Convert python array to num py array
    val_np_wordToVecArr = np.array(val_wordToVecArr)
    val_np_oneHotEncodingArr = np.array(val_oneHotEncodingArr)

    training = DnnTraining(input_dim=300, output_dim=3)
    training.startTraining(np_wordToVecArr, np_oneHotEncodingArr, val_np_wordToVecArr, val_np_oneHotEncodingArr, epochs=100)
    training.saveTrainedModel()

pipelineTest(train_word_arr, train_label_arr, val_word_arr, val_label_arr)

In [ ]:
def predict(test_data, test_label):
    model = load_model('dnn_model.h5')
    inference = DnnInference(test_data, test_label)
    inference.inferenceAndPrint(model, 20)